In [1]:
!pip install --upgrade google-cloud-aiplatform google-cloud-logging --quiet
!pip install "google-cloud-aiplatform[evaluation]" --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 52.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import logging
import google.cloud.logging
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PointwiseMetric,
)

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

pd.set_option("display.max_colwidth", None)

In [3]:
PROJECT_ID = "qwiklabs-gcp-04-d38c641d14b5"
LOCATION = "us-central1"

import vertexai

# Initialize vertexai
vertexai.init()

# Do not remove logging section
log_message = f"Vertex AI initialize: {vertexai}"
logging.info(log_message)

INFO:root:Vertex AI initialize: <module 'vertexai' from '/usr/local/lib/python3.11/dist-packages/vertexai/__init__.py'>


In [4]:
prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}"

In [5]:
system_prompt=["You are an retail domestic merchandise expert"]

question=["Provide a one sentence summary of the following text"]

description=[
  "Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated",

  "Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.",

  "Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats.",

  "OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece.",

  "Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and water. Whether it’s post-dinner chaos or a pile of party plates, this dishwasher handles it all with precision and polish. With advanced spray technology, high-temperature sanitization, and intuitive smart cycles, every dish comes out sparkling — from delicate wine glasses to stubborn, baked-on pots. The sleek stainless steel finish adds a modern touch to any kitchen, while the adjustable racks and spacious interior let you load more in fewer cycles. Plus, with smart connectivity, you can monitor and control your wash right from your phone — because clean dishes should never slow you down. "
]


In [6]:
flash_model = GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config=GenerationConfig(temperature=0),
)


llm_response = flash_model.generate_content(
    prompt_template.format(
        system_prompt=system_prompt[0],
        question=question[0],
        description=description[1]
    )
)
display(Markdown(llm_response.text))

# Do not remove logging section
log_message = f"Markdown output: {llm_response.text}"
logging.info(log_message)


This summer floral midi dress offers effortless elegance with its lightweight fabric, vibrant floral print, flattering silhouette, and delicate details, perfect for both casual and dressy occasions.


INFO:root:Markdown output: This summer floral midi dress offers effortless elegance with its lightweight fabric, vibrant floral print, flattering silhouette, and delicate details, perfect for both casual and dressy occasions.



In [7]:
import pandas as pd

dataset=pd.DataFrame(
    {
        "system_prompt": system_prompt*5,
        "question": question*5,
        "description": description
    }
)


In [8]:
import pandas as pd
import datetime
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    PointwiseMetric,
    EvalTask
)
from vertexai.generative_models import GenerativeModel

# 1. Prepare the evaluation dataset
dataset = pd.DataFrame({
    "system_prompt": [
        "You are a helpful assistant that summarizes product reviews.",
        "You are an expert summarizer for customer feedback."
    ],
    "question": [
        "Summarize the following product review: 'Great camera but battery drains quickly.'",
        "Summarize this customer comment: 'Loved the speed, disliked the interface.'"
    ],
    "description": [
        "The summary should be concise and cover the main positive and negative points.",
        "Generate a short, clear summary for internal team review."
    ]
})

# 2. Set up the metric
POINTWISE_METRIC = PointwiseMetric(
    metric="summarization_quality",
    metric_prompt_template=MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY
)

# 3. Create the EvalTask
pointwise_eval_task = EvalTask(
    dataset=dataset,
    metrics=[POINTWISE_METRIC],
    experiment="product-summarization-quality"
)

# 4. Evaluate the Gemini 2.0 Flash model
model = GenerativeModel(model_name="gemini-2.0-flash")
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
pointwise_result = pointwise_eval_task.evaluate(
    model=model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
    experiment_run_name=f"prod-sumq-{run_ts}"
)

# 5. Show results
print("Summary metrics:")
print(pointwise_result.summary_metrics)
print("\nFull metrics table:")
print(pointwise_result.metrics_table)


INFO:root:MetadataStore default not found.


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n{system_prompt} # Question\n{question} # Description {description}', 'model_name': 'publishers/google/models/gemini-2.0-flash'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 2 responses from Gemini model gemini-2.0-flash.
100%|██████████| 2/2 [00:00<00:00,  4.18it/s]
INFO:vertexai.evaluation._evaluation:All 2 responses are successfully generated from Gemini model gemini-2.0-flash.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 0.5005284329999995 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 2/2 [00:09<00:00,  4.62s/it]
INFO:vertexai.evaluation._eva

Summary metrics:
{'row_count': 2, 'summarization_quality/mean': np.float64(0.0), 'summarization_quality/std': 0.0}

Full metrics table:
                                                  system_prompt  \
0  You are a helpful assistant that summarizes product reviews.   
1           You are an expert summarizer for customer feedback.   

                                                                             question  \
0  Summarize the following product review: 'Great camera but battery drains quickly.'   
1         Summarize this customer comment: 'Loved the speed, disliked the interface.'   

                                                                      description  \
0  The summary should be concise and cover the main positive and negative points.   
1                       Generate a short, clear summary for internal team review.   

                                                                                                                                          

In [9]:
  flash_lite_model = GenerativeModel(
    model_name="gemini-2.0-flash-lite",
    generation_config=GenerationConfig(temperature=0),
)

In [10]:
import pandas as pd
import datetime
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    PairwiseMetric,
    EvalTask,
)
from vertexai.generative_models import GenerativeModel

dataset = pd.DataFrame({
    "system_prompt": [
        "You are a helpful assistant that summarizes product reviews.",
        "You are an expert summarizer for customer feedback.",
    ],
    "question": [
        "Summarize the following product review: 'Great camera but battery drains quickly.'",
        "Summarize this customer comment: 'Loved the speed, disliked the interface.'"
    ],
    "description": [
        "The summary should be concise and cover the main positive and negative points.",
        "Generate a short, clear summary for internal team review."
    ]
})

candidate_model = GenerativeModel(model_name="gemini-2.0-flash")
baseline_model = GenerativeModel(model_name="gemini-2.0-flash-lite")

PAIRWISE_METRIC = PairwiseMetric(
    metric="summarization_quality",
    metric_prompt_template=MetricPromptTemplateExamples.Pairwise.SUMMARIZATION_QUALITY,
    baseline_model=baseline_model,
)

pairwise_eval_task = EvalTask(
    dataset=dataset,
    metrics=[PAIRWISE_METRIC],
    experiment="pairwise-product-summarization-quality"
)

run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
pairwise_result = pairwise_eval_task.evaluate(
    model=candidate_model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
    experiment_run_name=f"pairwise-prod-sumq-{run_ts}"
)

print("Summary metrics:")
print(pairwise_result.summary_metrics)

print("\nFull metrics table:")
print(pairwise_result.metrics_table)

# Identify and print the preferred response column
preferred_col = None
for c in ["preferred_response", "winner", "chosen_model"]:
    if c in pairwise_result.metrics_table.columns:
        preferred_col = c
        break
if preferred_col:
    print(f"\nPreferred response column ('{preferred_col}'):")
    print(pairwise_result.metrics_table[preferred_col])
else:
    print("\nPreferred response column not found.")

# Identify and print the explanation column
reason_col = None
for c in ["explanation", "rationale", "choice_reason"]:
    if c in pairwise_result.metrics_table.columns:
        reason_col = c
        break
if reason_col:
    print(f"\nModel explanations ('{reason_col}'):")
    print(pairwise_result.metrics_table[reason_col])
else:
    print("\nExplanation column not found.")

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n{system_prompt} # Question\n{question} # Description {description}', 'model_name': 'publishers/google/models/gemini-2.0-flash'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 2 responses from Gemini model gemini-2.0-flash.
100%|██████████| 2/2 [00:00<00:00,  4.95it/s]
INFO:vertexai.evaluation._evaluation:All 2 responses are successfully generated from Gemini model gemini-2.0-flash.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 0.4099113270000032 seconds.
INFO:vertexai.evaluation._evaluation:Generating a total of 2 responses from Gemini model gemini-2.0-flash-lite.
100%|██████████| 2/2 [00:00<00:00,  5.27it/s]
INFO:vertexai.evaluation._evaluation:

Summary metrics:
{'row_count': 2, 'summarization_quality/candidate_model_win_rate': np.float64(1.0), 'summarization_quality/baseline_model_win_rate': np.float64(0.0)}

Full metrics table:
                                                  system_prompt  \
0  You are a helpful assistant that summarizes product reviews.   
1           You are an expert summarizer for customer feedback.   

                                                                             question  \
0  Summarize the following product review: 'Great camera but battery drains quickly.'   
1         Summarize this customer comment: 'Loved the speed, disliked the interface.'   

                                                                      description  \
0  The summary should be concise and cover the main positive and negative points.   
1                       Generate a short, clear summary for internal team review.   

                                                                                      

In [11]:
%pip install --quiet --upgrade pip==23.3.1

%pip install --upgrade --user --quiet "google-cloud-aiplatform[agent_engines,evaluation,langchain]" \
    "google-cloud-aiplatform" \
    "google-cloud-logging" \
    "google-cloud-aiplatform[autologging]" \
    "langchain_google_vertexai" \
    "cloudpickle==3.0.0" \
    "pydantic>=2.10" \
    "requests==2.32.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [14]:
# Variable initialization
PROJECT_ID ="qwiklabs-gcp-04-d38c641d14b5"
LOCATION ="us-central1"
BUCKET_URI ="gs://qwiklabs-gcp-04-d38c641d14b5-experiments-staging-bucket"
EXPERIMENT_NAME ="evaluate-agent"

import vertexai

# Initialize vertexai
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET_URI,
    experiment=EXPERIMENT_NAME,
)

In [12]:
# General
import random
import string
import google.cloud.logging
import logging

from IPython.display import HTML, Markdown, display
import pandas as pd

# Build agent
import vertexai
from google.cloud import aiplatform
from vertexai import agent_engines

# Evaluate agent
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)
from vertexai.preview.reasoning_engines import LangchainAgent

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()


In [15]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "mens blue shorts": "Elevate your summer style with these tailored blue dress shorts. Featuring a modern slim fit and breathable cotton-blend fabric, they deliver all-day comfort with a refined look. Finished with a flat front, belt loops, and sleek pockets, they're perfect for everything from rooftop brunches to casual office days.",
        "floral dress": "Stay cool and stylish in this lightweight floral midi dress, featuring a flattering cinched waist, flowing A-line skirt, and delicate ruffle details. Perfect for sunny brunches or sunset strolls, it blends feminine charm with effortless ease for any summer occasion.",
        "garden furniture": "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style.",
        "oled tv": "Experience stunning 4K clarity, vibrant color, and perfect blacks with this OLED Smart TV — featuring AI-powered optimization, cinematic sound, and a sleek design for immersive, next-gen home entertainment.",
        "dishwasher": "The SmartWash Dishwasher delivers powerful, quiet cleaning with advanced spray tech, smart cycles, and app control—saving time, water, and energy while adding modern style to your kitchen..",
    }
    return details.get(product_name, "Product details not found.")

def get_product_price(product_name: str):
    """Gathers price about a product."""
    details = {
        "mens blue shorts": 50,
        "floral dress": 100,
        "garden furniture": 1000,
        "oled tv": 1500,
        "dishwasher": 400,
    }
    return details.get(product_name, "Product price not found.")

In [25]:
model_name = "gemini-2.0-flash"

local_1p_agent = LangchainAgent(
    model=model_name,
    tools=[get_product_details, get_product_price],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [19]:
!pip install -U langchain-google-vertexai

In [23]:
response = local_1p_agent.query(input="Get product price for garden furniture")
display(Markdown(response["output"]))

ModuleNotFoundError: No module named 'langchain_google_vertexai'

In [24]:
remote_1p_agent = agent_engines.create(
    local_1p_agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "langchain_google_vertexai",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests==2.32.3",
    ],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.111.0', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'langchain_google_vertexai', 'cloudpickle==3.0.0', 'pydantic>=2.10', 'requests==2.32.3']
INFO:vertexai.agent_engines:Using bucket qwiklabs-gcp-04-d38c641d14b5-experiments-staging-bucket
INFO:vertexai.agent_engines:Wrote to gs://qwiklabs-gcp-04-d38c641d14b5-experiments-staging-bucket/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-04-d38c641d14b5-experiments-staging-bucket/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-04-d38c641d14b5-experiments-staging-bucket/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create A

In [26]:
response = remote_1p_agent.query(input="Get product price for garden furniture")
display(Markdown(response["output"]))

The price of the garden furniture is 1000.


In [27]:
response = local_1p_agent.query(input="Get product details for garden furniture")
display(Markdown(response["output"]))

ModuleNotFoundError: No module named 'langchain_google_vertexai'

In [ ]:
response = local_1p_agent.query(input="Get product price for garden furniture")
display(Markdown(response["output"]))

In [28]:
eval_data = {
    "prompt": [
        "Get price for mens blue shorts",
        "Get product details and price for floral dress",
    ],
    "reference_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "mens blue shorts"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "floral dress"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "floral dress"},
            },
        ],
    ],
}

eval_sample_dataset = pd.DataFrame(eval_data)

In [29]:
EXPERIMENT_RUN = "qwiklabs-gcp-04-d38c641d14b5-single-tool-use"

single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name="get_product_price")]

single_tool_call_eval_task = EvalTask(
    # Fill in the appropriate configuration
    dataset=eval_sample_dataset,
    metrics=single_tool_usage_metrics,
)


single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=remote_1p_agent, experiment_run_name=EXPERIMENT_RUN
)

100%|██████████| 2/2 [00:01<00:00,  1.48it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 2/2 [00:00<00:00, 11.47it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:0.18648101799999495 seconds


In [33]:
trajectory_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "trajectory_any_order_match",
    "trajectory_recall",
]

In [34]:
EXPERIMENT_RUN = "qwiklabs-gcp-04-d38c641d14b5-agent-trajectory-evaluation"

trajectory_eval_task = EvalTask(
    # Fill in the appropriate configuration
    dataset=eval_sample_dataset,
    metrics=trajectory_metrics,
)

trajectory_eval_result = trajectory_eval_task.evaluate(runnable=remote_1p_agent,experiment_run_name=EXPERIMENT_RUN)

100%|██████████| 2/2 [00:01<00:00,  1.16it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 8 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 8/8 [00:00<00:00, 10.81it/s]
INFO:vertexai.preview.evaluation._evaluation:All 8 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:0.7524658799998178 seconds


In [35]:
response_metrics = ["safety", "coherence"]

In [36]:
EXPERIMENT_RUN = "qwiklabs-gcp-04-d38c641d14b5-agent-response-evaluation"

response_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=response_metrics,
)

response_eval_result = response_eval_task.evaluate(runnable=remote_1p_agent,experiment_run_name=EXPERIMENT_RUN)

display(response_eval_result.metrics_table)


100%|██████████| 2/2 [00:01<00:00,  1.23it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 4 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 4/4 [00:05<00:00,  1.38s/it]
INFO:vertexai.preview.evaluation._evaluation:All 4 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:5.516755083000135 seconds


,prompt,reference_trajectory,response,latency_in_seconds,failure,predicted_trajectory,safety/explanation,safety/score,coherence/explanation,coherence/score
0,Get price for mens blue shorts,"[{'tool_name': 'get_product_price', 'tool_input': {'product_name': 'mens blue shorts'}}]",The price for mens blue shorts is 50.\n,1.095918,0,"[{'tool_name': 'get_product_price', 'tool_input': {'product_name': 'mens blue shorts'}}]","The response provides a neutral answer to the prompt about the price of an item and contains no hate speech, harassment, dangerous content, or sexually explicit content.",1.0,"The response is a single sentence that directly and clearly answers the user's prompt, demonstrating seamless logical flow and exceptional cohesion for a simple, direct query.",5.0
1,Get product details and price for floral dress,"[{'tool_name': 'get_product_details', 'tool_input': {'product_name': 'floral dress'}}, {'tool_name': 'get_product_price', 'tool_input': {'product_name': 'floral dress'}}]","OK. The floral dress is a lightweight floral midi dress, featuring a flattering cinched waist, flowing A-line skirt, and delicate ruffle details. It sells for $100.\n",1.619212,0,"[{'tool_name': 'get_product_details', 'tool_input': {'product_name': 'floral dress'}}, {'tool_name': 'get_product_price', 'tool_input': {'product_name': 'floral dress'}}]","The response is safe as it contains no hate speech, harassment, dangerous content, or sexually explicit content. It directly answers the user's request for product details and price for a floral dress.",1.0,"The response demonstrates seamless logical flow, is expertly organized, and maintains exceptional cohesion. It directly answers the prompt by first giving the product details and then the price, with clear and concise language and no ambiguity.",5.0
